In [1]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import sklearn
import os
import cv2
from tqdm import tqdm

In [ ]:
def convertjpg(jpgfile,outdir,width=64,height=64):
    img=Image.open(jpgfile)   
    new_img=img.resize((width,height),Image.BILINEAR)   
    new_img.save(os.path.join(outdir,os.path.basename(jpgfile)))
for jpgfile in glob.glob('/root/autodl-tmp/thispersondoesnotexist.10k/*.jpg'):
    convertjpg(jpgfile,'/root/autodl-tmp/thispersondoesnotexist.10k')
for jpgfile in glob.glob('/root/autodl-tmp/0/*.jpg'):
    convertjpg(jpgfile,'/root/autodl-tmp/0')

In [2]:
def getFileName(filepath):
    path_list = []
    pathList = os.listdir(filepath)
    for allFile in pathList:
        everyFile = os.path.join(filepath, allFile)
        path_list.append(everyFile)
    return path_list
training_path = getFileName('/root/autodl-tmp/thispersondoesnotexist.10k')
testing_path = getFileName('/root/autodl-tmp/0')

In [3]:
training_path = training_path[:10000]

In [4]:
testing_img_path = training_path[:800] + testing_path[:200]
training_img_path = training_path[800:]

In [5]:
test_y = 800*[0] + 200*[1]
test_y = np.array(test_y)

In [6]:
train = np.array([cv2.imdecode(np.fromfile(path,dtype=np.uint8),-1)/ 255. for path in training_img_path])
train = train.reshape(9200,64,64,3)
print(train.shape)
test = np.array([cv2.imdecode(np.fromfile(path,dtype=np.uint8),-1)/ 255. for path in testing_img_path])
test = test.reshape(1000,64,64,3)
print(test.shape)

(9200, 64, 64, 3)
(1000, 64, 64, 3)


In [7]:
train_neg = [t.reshape(t.shape[0]* t.shape[1] * t.shape[2]) for t in train]
test_tmp = [t.reshape(t.shape[0]* t.shape[1] * t.shape[2]) for t in test]

In [8]:
def kmeansClustering(X,Y,nClusters):#K-means algorithm 
    from sklearn.cluster import KMeans
    KMEANS = KMeans(n_clusters = nClusters, init = 'random',max_iter = 300,n_init = 10,random_state = 0)
    kmeans = KMEANS.fit(X)
    Z = kmeans.labels_
    inertia = KMEANS.inertia_
    #Kmeans Results
    kmeansR = pd.crosstab(Y,Z)
    maxVal = kmeansR.idxmax()
    return Z,kmeansR,maxVal,inertia

In [9]:
Z,kmeansR,maxVal,inertia = kmeansClustering(test_tmp, test_y, 2)

In [10]:
kmeansR

col_0,0,1
row_0,,
0,787,13
1,0,200


In [11]:
TP = 200
TN = 787
FP = 13
FN = 0

In [17]:
from sklearn.metrics import roc_auc_score
acc = (TP+TN)/1000
recall = TP/(TP+FN)
precision = TP/(TP+FP)
f1 = 2*precision*recall/(precision+recall)
Z = Z.reshape(1000,1)
test_y = test_y.reshape(1000,1)
auc_score = roc_auc_score(Z,test_y)
print('accuracy: %.3f, recall: %.3f, precision: %.3f, related f1 score: %.3f, AUC: %.3f'%(acc,recall,precision,f1,auc_score))
print('In %d data of test set, TP: %d, FN: %d, FP: %d, TN: %d'%(len(test_y),TP,FN,FP,TN))

accuracy: 0.987, recall: 1.000, precision: 0.939, related f1 score: 0.969, AUC: 0.969
In 1000 data of test set, TP: 200, FN: 0, FP: 13, TN: 787
